1. Imports e Leitura dos Arquivos

In [1]:
import pandas as pd

In [2]:
atividade_01 = pd.read_excel('atividade01.xlsx')
atividade_01

,Aluno,Turma,Nota
0,Jéssica,A,10
1,Pedro,A,8
2,Maria,B,9
3,José,C,7


In [3]:
atividade_02 = pd.read_excel('atividade02.xlsx')
atividade_02

,Aluno,Turma,Nota
0,Jéssica,A,9
1,Pedro,A,8
2,Maria,B,7
3,Marcos,D,5


2. Alunos que realizaram as duas atividades

In [4]:
atividade_completa = atividade_01.merge(atividade_02[['Aluno', 'Nota']], 
                                        how='inner', on='Aluno',
                                        suffixes=('_Atividade_01', '_Atividade_02'))

In [5]:
atividade_completa

,Aluno,Turma,Nota_Atividade_01,Nota_Atividade_02
0,Jéssica,A,10,9
1,Pedro,A,8,8
2,Maria,B,9,7


3. Alunos que realizaram uma única atividade

In [6]:
from pandasql import sqldf

consulta = '''  SELECT 
                    COALESCE(t1.Aluno, '-') AS Aluno_Atividade_01, 
                    COALESCE(t2.Aluno, '-') AS Aluno_Atividade_02, 
                    t1.Nota AS Atividade_01, 
                    t2.nota AS Atividade_02
                FROM 
                    atividade_01 t1
                FULL OUTER JOIN 
                    atividade_02 t2
                ON 
                    t1.Aluno = t2.Aluno 
                WHERE 
                    t1.Aluno IS NULL OR t2.Aluno IS NULL'''

atividade_unica = sqldf(consulta, locals()).fillna('-')

In [7]:
atividade_unica

,Aluno_Atividade_01,Aluno_Atividade_02,Atividade_01,Atividade_02
0,José,-,7.0,-
1,-,Marcos,-,5.0


4. FULL OUTER JOIN via ``.merge()` com identificação do arquivo de origem

In [9]:
atividade_resumo = atividade_01.merge(atividade_02[['Aluno', 'Nota']], 
                                    how='outer', on='Aluno',
                                    suffixes=('_Atividade_01', '_Atividade _02'),
                                    indicator=True)

In [10]:
atividade_resumo

,Aluno,Turma,Nota_Atividade_01,Nota_Atividade _02,_merge
0,Jéssica,A,10.0,9.0,both
1,Pedro,A,8.0,8.0,both
2,Maria,B,9.0,7.0,both
3,José,C,7.0,NaN,left_only
4,Marcos,NaN,NaN,5.0,right_only


In [12]:
df = pd.concat([atividade_01, atividade_02])
df

,Aluno,Turma,Nota
0,Jéssica,A,10
1,Pedro,A,8
2,Maria,B,9
3,José,C,7
0,Jéssica,A,9
1,Pedro,A,8
2,Maria,B,7
3,Marcos,D,5
